In [1]:
from environment import Environment
from magiccube import Cube
from agents import A2C, DQN, PPO
import yaml
import numpy as np
import itertools
import torch
import torch.nn as nn

In [2]:
args = yaml.safe_load(open("config.yaml", "r"))
phase = "f2l"

env = Environment(
    phase=phase,
    args=args["environment"]
)

agent = DQN(env, phase, args)

In [3]:
cube = Cube()
print(env.cube)

          W  B  O                   
          W  W  R                   
          R  B  B                   
 R  O  B  W  W  R  Y  W  Y  G  O  G 
 G  O  G  O  G  W  G  R  R  B  B  R 
 W  O  Y  B  G  R  Y  R  O  B  B  O 
          O  Y  G                   
          Y  Y  Y                   
          G  Y  W                   



In [4]:
env.cube.rotate(cube.generate_random_moves())
print(env.cube)

          G  O  W                   
          Y  W  W                   
          R  W  G                   
 O  G  Y  B  B  Y  R  O  G  R  Y  W 
 B  O  R  G  G  W  G  R  R  Y  B  R 
 Y  B  O  B  W  B  R  O  Y  O  G  O 
          W  R  W                   
          Y  Y  B                   
          B  O  G                   



In [5]:
env.scramble()
#obs, reward, done = env.step("B'")
print(env.state2) 
#print(env.state2.shape)
#print(env.algorithm.status())
#print(env.cube)
#print(agent.policy_net(agent.state_to_tensor(env.state2)))

[[[ 4  0  2 23  1  3  4  0]
  [ 1  3 -1  1  1  3 -1  1]
  [ 4  3  1  0  1  3  5  2]
  [ 4 -1  2 14  1 -1  4  3]
  [ 1 -1 -1  4  1 -1 -1  4]
  [ 5 -1  1  5  1 -1  5  5]
  [ 1  5  2  8  1  2  4  6]
  [ 5  0 -1 22  1  2 -1  7]
  [ 2  1  4  6  1  2  5  8]
  [-1  3  4  9 -1  3  4  9]
  [-1  3 -1 10 -1  3 -1 10]
  [-1  3  5 11 -1  3  5 11]
  [-1 -1  4 12 -1 -1  4 12]
  [-1 -1  5 13 -1 -1  5 13]
  [-1  5  2 16 -1  2  4 14]
  [-1  2 -1 15 -1  2 -1 15]
  [-1  4  0 20 -1  2  5 16]
  [ 2  5  0 25  0  3  4 17]
  [ 0  3 -1 18  0  3 -1 18]
  [ 0  4  3 17  0  3  5 19]
  [ 0 -1  2 24  0 -1  4 20]
  [ 0 -1 -1 21  0 -1 -1 21]
  [ 4 -1  1  3  0 -1  5 22]
  [ 0  3  5 19  0  2  4 23]
  [ 1  2 -1  7  0  2 -1 24]
  [ 5  1  3  2  0  2  5 25]]]


In [6]:
env.scramble()
print(env.cube)

          O  B  O                   
          G  W  W                   
          R  O  O                   
 Y  W  G  Y  B  Y  G  R  W  B  R  B 
 G  O  W  B  G  W  O  R  R  G  B  O 
 O  O  B  Y  G  B  R  R  W  R  B  G 
          R  Y  W                   
          Y  Y  Y                   
          W  Y  G                   



In [110]:
state_tensor = torch.from_numpy(env.state2).to("cuda:0").float()
state_tensor.shape

torch.Size([1, 26, 8])

In [ ]:
class RubikAttention(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.attention = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=dim,
            batch_first=True
        )

        self.fc = nn.Sequential(
            nn.Linear(dim, dim * 2),
            nn.ReLU(inplace=True),
            nn.Linear(dim * 2, dim * 2),
            nn.ReLU(inplace=True),
            nn.Linear(dim * 2, dim)
        )

    def forward(self, x):
        x, _ = self.attention(x, x, x)
        x = self.fc(x)

        return x

class RubikNet(nn.Module):
    def __init__(
        self,
        num_layers: int,
        output_dim: int
    ) -> None:
        super().__init__()
        self.attn_layers = nn.Sequential(
            *[RubikAttention(2) for _ in range(num_layers)]
        )

    def forward(self, x):
        x = self.attn_layers(x)

        return x

In [317]:
net =RubikNet(4).to("cuda")

net(state_tensor).shape

torch.Size([1, 26, 8])

In [120]:
# Definiamo un layer lineare: input con 3 feature, output con 2 feature
fc1 = nn.Linear(in_features=3, out_features=256)
fc2 = nn.Linear(in_features=256, out_features=2)

# Esempio di input: batch di 4 esempi, ciascuno con 3 feature
input_matrix = torch.tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0]
])  # shape: (4, 3)

# Passaggio della matrice al layer
output = fc1(input_matrix)
print(output)
output = fc2(output)

print("Output:", output)

tensor([[ -2.7835,   1.3881,   1.3501,  ...,   0.7526,  -0.0367,   0.5035],
        [ -7.0616,   3.6675,   2.3473,  ...,   1.7061,   0.4502,   0.9719],
        [-11.3397,   5.9469,   3.3445,  ...,   2.6596,   0.9370,   1.4404],
        [-15.6178,   8.2263,   4.3417,  ...,   3.6132,   1.4239,   1.9088]],
       grad_fn=<AddmmBackward0>)
Output: tensor([[0.2159, 0.9814],
        [0.7017, 2.0261],
        [1.1874, 3.0708],
        [1.6731, 4.1155]], grad_fn=<AddmmBackward0>)
